# Lecture 8: More about Monte Carlo methods

## Physics 7810, Spring 2020

## 8.1 - Trial moves
 
For lattice models such as the Ising model, simple discrete Monte Carlo moves such as spin flips can be used. It's easy to verify that such moves satisfy detailed balance, as the proposal probability $\alpha$ is manifestly symmetric,

$$
\alpha(o \to n) = \alpha(n \to o).
$$

In other words, the probability of proposing to flip a spin in a given MC move is equal to the probability of proposing to 'unflip' it in the next MC move.

It can be more difficult to design trial moves that preserve detailed balance in more complex situations, e.g., for off-lattice models such as the Lennard-Jones or hard sphere systems.

## 8.2 - Translational moves
 
For off-lattice models, it's conventional to generate trial particle displacements within a cubic volume of edge length $\Delta$ centered on the current particle position,

$$
\begin{eqnarray}
x_i^\prime & = & x_i + \Delta \left[ \mathrm{unif}(0,1) - 1/2 \right] \\
y_i^\prime & = & y_i + \Delta \left[ \mathrm{unif}(0,1) - 1/2 \right] \\
z_i^\prime & = & z_i + \Delta \left[ \mathrm{unif}(0,1) - 1/2 \right],
\end{eqnarray}
$$

where $\mathrm{unif}(0,1)$ denotes a continuous random variable uniformly distributed on the interval $(0,1)$.

It's clear that the reverse trial move is equally probable, so this choice satisfies the detailed balance requirement.  A more symmetric choice would be to generate displacements within a *spherical* volume centered on the current particle position, but this procedure isn't commonly used. Why not?

This trial move involves a free parameter $\Delta$, which can be adjusted to optimize the MC procedure, as we will discuss shortly.

In a conventional translational trial move, state $n$ is obtained from state $o$ by generating a particle displacement  anywhere within the shaded region:

<img src="images/A&T_Fig_4.2.png" alt="Drawing" style="width: 500px;">

Figure from *Computer Simulation of Liquids*, by Mike Allen and Dominic Tildesley.

How do we gauge the efficiency of a given MC algorithm (e.g., as a function of $\Delta$)?

We can generally define the efficiency of a method as:

$$
\mathrm{efficiency} = \frac{\mathrm{Number\ of\ statistically\ independent\ configurations\ generated}}{\mathrm{CPU\ time}}.
$$

Note that this definition applies equally well to MD and MC simulations. Making use of this definition requires understanding what we mean by *statistically independent*, and developing ways of measuring the number of statistically independent samples generated in a given simulation.

Also, note that a simulation procedure may be optimal for sampling one quantity, but non-optimal for sampling another. For example, an algorithm for simulating non-spherical molecules may be optimal for sampling *translational* degrees for freedom, but not for sampling *orientational* degrees of freedom.

*Collective* MC moves that involve updating many degrees of freedom simultaneously are usually (but not always) highly inefficient, because most such trial moves will be rejected.

As described in Frenkel and Smit, a crude but generally effective way of optimizing translational trial moves with respect to $\Delta$ involves estimating the rate at which statistically independent samples are generated by measuring a quantity related to the mean-squared particle displacement (a measure of the rate at which new configurations are explored).

The mean-squared displacement of a particle in a time interval $\tau$ is defined as

$$
\left\langle \Delta {\bf r}_i^2(\tau)  \right\rangle
= \left\langle \left[ {\bf r}_i(t + \tau) - {\bf r}_i(t) \right]^2 \right\rangle,
$$

where the angle brackets denote an average over all particles $i$ and all time origins $t$. In a fluid phase in which particles can diffuse, the mean-squared displacement is related to the diffusion constant $D$ by

$$
\left\langle \Delta {\bf r}_i^2(\tau)  \right\rangle = 2 d D \tau
$$

for large $\tau$, where $d$ is the spatial dimensionality. In the absence of diffusion, for example in a crystalline or glassy solid, this isn't a useful measure of sampling efficiency. For this purpose we need to define slightly different quantity, and we need to clarify what we mean by 'time' in an MC simulation.

To estimate the sampling efficiency of an MC algorithm, we compute an $\mathrm{msd}$-like quantity, the single-move mean-squared particle displacements averaged over MC moves,  

$$
\left\langle \Delta {\bf r}_i^2 \right\rangle,
$$

where the angle brackets now denote an average over all particles and over all trial MC moves. Note that only *accepted* moves make a nonzero contribution to this average, but the average is over *all* trial moves (accepted or not). Thus, $\left\langle \Delta {\bf r}_i^2 \right\rangle$ will be small for small $\Delta$ (where most trial moves are accepted, but only generate small displacements) and for large $\Delta$ (where most trial moves are rejected).

We can now define a diffusion-constant-like measure of efficiency based on $\left\langle \Delta {\bf r}_i^2 \right\rangle$, namely

$$
\mathrm{efficiency} = \frac{\left\langle \Delta {\bf r}_i^2 \right\rangle}{\mathrm{CPU\ time}}.
$$

This measure of efficiency can be used for 'on-the-fly' optimization of MC simulations, although such optimizations shouldn't be carried out during *production* runs, as they typically (weakly) violate detailed balance.

Qualitative dependence of the single-move mean-squared displacement on the average size $\Delta$ of trial moves (left), and dependence of the computational cost per move on $\Delta$ for continuous and hard-core potentials (right). For hard-core systems, an 'early exit' strategy leads to a decrease in computational cost with increasing $\Delta$.
 
<img src="images/Frenkel_Fig_3.4.png" alt="Drawing" style="width: 1000px;">

Figure from *Understanding Molecular Simulation: from Algorithms to Applications*, by Daan Frenkel and Berend Smit.

## 8.3 - Long-range corrections
 
Truncation of pair interactions using cutoffs introduces a dependence of the simulation results on the interaction cutoff $r_c$. It's usually highly desirable to remove this artifical dependence on $r_c$ if at all possible.

To a first approximation, we can do this by computing long-range corrections to average thermodynamic quantities such as the potential energy and pressure, assuming that the density of interacting pairs is uniform beyond the interaction cutoff $r_c$